# Intro to APIs
(connecting with other apps)

API: Application Program(ming) Interface
  * Interfaces for code/computers to talk to each other
  * Any set of tools, code,  routines that can be used in an application.
  
## WEB APIs (the ones we care about)
  * Generally communicate via HTTP
      * Make an HTTP requests to an API
      * Get some data back!
  * Usually returns JSONs
  * APIs don't respond with HTML
      * HTML contains structure of page. APIs respond w/ data, not structure.
      
  * They use simpler data formats like [XML](https://www.w3schools.com/xml/xml_whatis.asp) and [JSON](https://developers.squarespace.com/what-is-json/)
  
  **Endpoint:** Term for urls of APIs, basically where the API call will be made.

###  XML: Extended Markup Language
 * Syntactically similar to HTML, but doesn't describe presentation/structure like HTML does.
 * Example: 
 ```xml

    <person>
        <age>21</age>
        <name></name>
        <city>Los Angeles</city>
    </person>
 ```
 
### JSON: JavaScript Object Notation
* Pretty **GOTDANG** similar to JavaScript, only the "keys" need quotes around them.
* Example:

  ```json

   {
     "person": {
        "age": "21",,
        "name":"Travis",
        "city": "Los Angeles"
     }
   }
    
  ```

### (BONUS from Steeve) [YAML](https://en.wikipedia.org/wiki/YAML): YAML Ain't Markup Language

* Resources:
  * [Complete idiot's introduction to yaml](https://github.com/Animosity/CraftIRC/wiki/Complete-idiot%27s-introduction-to-yaml), 
 
  * [YAML start page](http://yaml.org/start.html), 
  * [Learn YAML in Y Minutes](https://learnxinyminutes.com/docs/yaml/), 
  * [YAML Syntax](http://docs.ansible.com/ansible/latest/reference_appendices/YAMLSyntax.html)
  
* Aims to be simpler than even JSON
* Example:
  ```yaml
  person
      - name: "Travis"
      - age: 21
      - city: "Los Angeles"
  ```



# Making Requests Programmatically


## Request (Node Package)
* To make a request from a javascript file, use the [`request`](https://github.com/request/request) Node package.
* remember to npm install request
* syntax: 
    ```javascript
    var request = require('request');
    
    // caution: body often comes back as a string. We usually need JSON 
    //   (or XML if you're old)
    request('https://google.com', function (error, response, body) {

      if(!error && response.statusCode === 200){

        // turn body (which is returned as a string) into a JSON object.
        var thing = JSON.parse(body);
        console.log(thing["query"]["results"]["channel"]["astronomy"]["sunset"]);
      }

      else {
        // Print the error if one occurred
        console.log('error:', error);

        // Print the response status code if a response was received
        console.log('statusCode:', response && response.statusCode);
      }
    });
  ```

[Commmon HTTP Status Codes](https://www.smartlabsoftware.com/ref/http-status-codes.htm)

  * 200: OK
  * 300: Multiple Choices
  * 301: Moved Permanently
  * 302: Found
  * 304: Not Modified
  * 307: Temporary Redirect
  * 400: Bad Request
  * 401: Unauthorized
  * 403: Forbidden
  * 404: Not Found
  * 410: Gone
  * 500: Internal Server Error
  * 501: Not Implemented
  * 503: Service Unavailable
  * 550: Permission denied 




### Making Requests That Handle JSON



Sidenote:
* `curl`: **C**ommand-line **URL** interface
  * Syntax: `curl http://link.to/webpage`
  * returns the same content a browser would, without formatting/rendering.

### Example of a full API-calling app.

### In app.js
  ```javascript
    // set up app with express.
    var express = require("express");
    var app = express();

    // include request
    var request = require("request");

    // tell express to render ejs files (in the "views/" folder).
    app.set("view engine", "ejs");

    // default homepage is the search form
    app.get("/", function(req, res){
      res.render("search");
    });

    // render the page after calling the API to search for 
    // the "name" field in the form in the search.ejs file.
    app.get("/results", function(req, res){

      // pull out the query.
      var query = req.query.search;

      // note the "?". This lets us know the user is querying something
      //  so to access that thing will be req.query...
      // the s after the "?"(query string) stands for "search".
      var url = "http://www.omdbapi.com/?s="+ query + "&apikey=thewdb";

      request(url, function(error, response, body){
        if (!error && response.statusCode == 200) {
          var data = JSON.parse(body);
          // console.log(body);

          // render the results.ejs file, pass in the data Object 
          // REMEMBER: {data <- data}
          res.render("results", {data:data});
          }
      });
    });

    // run server!
    app.listen(process.env.PORT, process.env.IP, function(req, res) {
      console.log("Movie app started!");
    })
  ```

#### In search.ejs (acting as the homepage)
  ```ejs
    <h1>Search for a movie!</h1>

    <!--GET the "/results" route, specified with the query held in "search"-->
    <form action="/results" method="GET">

      <!--name is what the "/result" route will search for-->
      <input type="text" name="search" placeholder="search term">

      <!--just a reg submit button-->
      <input type="submit" placeholder>
    </form>

  ```

#### In results.ejs
  ```ejs

    <h1>Results Page!</h1>

    <ul> 

    <!--for each result (movie) in the "Search" array in the data JS Object,-->
    <!--"print" the title and year of the result as an li.-->
    <% data["Search"].forEach(function(movie){  %>
     <li><i><%= movie["Title"]%></i> - <strong><%= movie["Year"]%></strong></li>
    </ul>
    <% }); %>

    <!--Back button, leading us to the root route.-->
    <a href="/">Search Again</a>
  ```
  
### App's flow:
1. User browses to "/", the root route is run, serving search.ejs<br><br>

2. When the form is submitted in search.ejs, the user is directed to "/results" (GET METHOD) with the "name" (which contains the value "search") containing the search term.<br><br>

3. In app.js, the "/results" route is run.<br>
  1. the search term is pulled from the request's query (`var query = req.query.search`)<br><br>

  2. the full url is made from the query string + `query` + the apikey<br><br>
  
  3. we request the url (calling the api with the query)<br><br>
  
  4. if status code is 200 (OK):<br>
      
     1. parse the body into JSON<br><br>
      
     2. send the parsed JSON to the results.ejs<br><br>
      
     3. for each result in the "Search" (results) array:<br>
        
         1. print the reults title, and year.<br><br>
      
    4. Optionally, link back to the homepage (the search page).